In [1]:
import numpy as np
from numpy.linalg import eigvals
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
import pandas as pd

In [3]:
#
#  extend path by location of the dvr package
#
import sys
sys.path.append('../../Python_libs')
import dvr
from jolanta import Jolanta_3D
from captools import simple_traj_der

In [3]:
amu_to_au=1822.888486192
au2cm=219474.63068
au2eV=27.211386027
Angs2Bohr=1.8897259886

In [4]:
#
# Jolanata-3D parameters a, b, c: (0.028, 1.0, 0.028)
#
#   CS-DVR:   
#      bound state: -7.17051 eV
#      resonance (3.1729556 - 0.16085j) eV
#
jparam=(0.028, 1.0, 0.028)

In [5]:
#
#  compute DVR of T and V
#  then show the density of states
#  in a potential + energy-levels plot
#
xmax=30
xmin=0   # grid from -xmin to xmax
thresh = 7   # maximum energy for state in the plot in eV
ppB = 15     # grid points per Bohr

nGrid=int((xmax-xmin)*ppB)
xs = dvr.DVRGrid(xmin, xmax, nGrid)
Ts = dvr.KineticEnergy(1, xmin, xmax, nGrid)
Vs=Jolanta_3D(xs, jparam)
[energy, wf] = dvr.DVRDiag2(nGrid, Ts, Vs, wf=True)

n_ene=0
for i in range(nGrid):
    print("%3d  %12.8f au = %12.5f eV" % (i+1, energy[i], energy[i]*au2eV))
    n_ene += 1
    if energy[i]*au2eV > thresh:
        break

# "DVR normalization", sum(wf[:,0]**2)
# this is correct for plotting

c=["orange", "blue"]
h=float(xmax) / (nGrid+1.0)
scale=400

plt.plot(xs,Vs*au2eV, '-', color="black")
for i in range(n_ene):
    plt.plot(xs, scale*wf[:,i]**2+energy[i]*au2eV, '-', color=c[i%len(c)])
#plt.ylim(0, 20)
plt.xlabel('$x$ [Bohr]')
plt.ylabel('$E$ [eV]')
plt.ylim(-8,8)
plt.show()

  1   -0.26351131 au =     -7.17051 eV
  2    0.01683168 au =      0.45801 eV
  3    0.05606363 au =      1.52557 eV
  4    0.10555774 au =      2.87237 eV
  5    0.12662372 au =      3.44561 eV
  6    0.18685288 au =      5.08453 eV
  7    0.26221360 au =      7.13520 eV


In [30]:
""" complex diagonalization example """

theta=25.0/180.0*np.pi
print("theta = %f" % (theta))
Vs = Jolanta_3D(xs*np.exp(1j*complex(theta)), jparam)
H_theta = np.exp(-2j*complex(theta)) * Ts + np.diag(Vs)
energies = eigvals(H_theta)
energies.sort()
energies[:20]*au2eV
plt.cla()
plt.plot(energies.real*au2eV, energies.imag*au2eV, 'o')
plt.xlim(-8,10)
plt.ylim(-2,0.5)

theta = 0.436332


(-2, 0.5)

For small $\theta$ everything is fine, but for larger angles basis set artifacts become apparent = there are eigenvalues off the rotation string with small real and hige imaginary part. Thus, above some $\theta$, sorting the eigenvalue by low real part stops working as a filter. 

In [31]:
#
#  complex scaling loop: 
#
#  start on the real axis (theta=0) and rotate to theta = theta_max 
#
#  we keep n_keep energies with the lowest real part 
#

n_keep=30
theta_min=0
theta_max=40
n_theta=theta_max - theta_min + 1
thetas=[t for t in range(theta_min, theta_max+1)]

run_data = np.zeros((n_theta,n_keep), complex)  # array used to collect all theta-run data
#run_data = np.zeros((n_keep,n_theta), complex)  # array used to collect all theta-run data

for i_theta in range(n_theta):
    theta=thetas[i_theta]/180.0*np.pi
    Vs = Jolanta_3D(xs*np.exp(1j*complex(theta)), jparam)
    H_theta = np.exp(-2j*complex(theta)) * Ts + np.diag(Vs)
    energies = eigvals(H_theta)
    energies.sort()
    run_data[i_theta,:] = energies[0:n_keep]
    #run_data[:,i_theta] = energies[0:n_keep]
    print(i_theta+1, end=" ")
    if (i_theta+1)%10==0:
        print()

run_data *= au2eV

1 2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 40 
41 

Plot the raw data; use matplotlib to zoom.

In [32]:
plt.cla()
for i in range(0, n_keep):
    plt.plot(run_data[:,i].real,  run_data[:,i].imag, 'o')
plt.xlim(-1,10)
plt.ylim(-1,0)
plt.show()

Two follow ideas:

1. at the last $\theta$ compute the angles and compare with $2*\theta$ if significantly smaller, then resonance
2.  once a trajectory has five values, use them to establish a Pade[2,2] then predict for the next theta



In [33]:
def follow_nearest(follow, es):
    """
    follow the energy closet to e0 from the real axis into the complex plane
    es is a table of theta-run data es[i_theta,j_energies]
    the algorithm used is simply nearest to the old energy
    """
    (n_thetas, n_energies) = es.shape
    trajectory = np.zeros(n_thetas,complex)
    for j in range(0,n_thetas):
        i = np.argmin(abs(es[j,:]-follow))
        follow = trajectory[j] = es[j,i]
    return trajectory

In [34]:
n_save = n_keep//2
trajectories = np.zeros((n_theta, n_save), complex)
for j in range(n_save):
    trajectories[:,j] = follow_nearest(run_data[0,j], run_data)
plt.xlim(-1,20)
plt.ylim(-10,0)
for i in range(0, n_save):
    plt.plot(trajectories[:,i].real,  trajectories[:,i].imag, '-')
plt.show()

In [35]:
#
#  save n_save trajectories to file
#  csv as real and imag 
#  (at the moment easier than csv with complex)
#  also, include no header, because the energies need to be sorted
#  into trajectories first
#
#fname="complex_scaling_rmax."+str(int(rmax))+"_ppB."+str(ppB)+".csv"
#read_write.write_theta_run(fname,thetas,trajectories)

The resonance clearly stands out in CS/DVR.
Follow the trajectory that finishes with the smallest angle.

In [36]:
for i in range(n_save):
    print(i, np.angle(trajectories[-1,i],deg=True))
print("Minimum angle index:",np.argmax(np.angle(trajectories[-1,0:n_save])))

0 -179.99980238441225
1 -105.68845920485826
2 -114.15405497496475
3 -114.46044686754207
4 -2.9020156944302453
5 -98.1668419877256
6 -101.11743002992317
7 -32.10968523480819
8 -98.4560292763116
9 -98.01144437707109
10 -52.29813241727825
11 -96.33500132733968
12 -92.28516799540056
13 -63.17563466846886
14 -90.97816746794189
Minimum angle index: 4


In [37]:
res_traj = trajectories[:,4]
abs_der = np.abs(simple_traj_der(thetas, res_traj))
plt.cla()
plt.plot(thetas, np.log(abs_der), 'o-')
plt.show()

In [38]:
#
#  get a feeling for the stabilitity of the value
#
j_opt = np.argmin(abs_der)
if j_opt == len(thetas)-1:
    j_opt -= 1
print(j_opt, thetas[j_opt])
print('resonance', res_traj[j_opt])
print('abs_der', abs_der[j_opt])
print('change down', res_traj[j_opt-1]-res_traj[j_opt])
print('change up', res_traj[j_opt+1]-res_traj[j_opt])

34 34
resonance (3.1729556212701966-0.16084889558781004j)
abs_der 1.4360534078713517e-06
change down (-1.4354245561065682e-06+2.766522097519264e-08j)
change up (1.4358934654623567e-06+6.092757803832782e-08j)


In [27]:
len(thetas)

41